In [324]:
import numpy as np
import re
import lda
import lda.datasets
import pandas as pd
from scipy.sparse import coo_matrix
import math as mt
from nltk.corpus import stopwords

In [336]:
doc_data = pd.read_csv('output_100-300_230.csv', header=None, low_memory=False)


In [337]:

LDA_data = doc_data.drop(0, axis=1)
LDA_data_docs = LDA_data.iloc[:,1:]


In [338]:
test = LDA_data_docs.values.tolist()
documents = [[i for i in j if (type(i) == str )] for j in test]

In [339]:
# get numerical category labels for each subreddit

In [340]:
# make keys for each doc in list

K = list(doc_data.iloc[0:LDA_data_docs.shape[0],0])
idx = list(range(LDA_data_docs.shape[0]))
K = [i + str(j) for i, j in zip(K, idx)]

D=dict.fromkeys(K)
D.update(zip(K,documents))
docs = D

In [341]:
#
# start here!
#

In [342]:
#build master dataframe for final output

n_top_words = 10
model = lda.LDA(n_topics=2, n_iter=50, random_state=1, alpha = 25, eta=0.1)

# a= np.array([0])
# b = np.array(range(n_top_words))
# c = np.concatenate([a,b,b])
# output_df = pd.DataFrame(columns=c)

In [343]:
# break things up into a arrays of size doc_len
# portions of this code adapted from: 
#https://datascience.blog.wzb.eu/2016/06/17/creating-a-sparse-document-term-matrix-for-topic-modeling-via-lda/

iteration = 0

for postname, val in docs.items():

    #
    # break up doc into subdocs
    #
    
    print("ITERATION __________________________: ", iteration)
    doc_len = 30
    val = np.array(val)

    # get uneven part length
    doc_end = val[-(val.shape[0] % doc_len):]
    new_doc = val[0:-(val.shape[0] % doc_len)]
    new_doc = np.reshape(new_doc, (doc_len,-1))


    #convert to list
    new_doc = new_doc.tolist()
    # append last block to list
    new_doc.append(doc_end.tolist())
    
    # make integer keys for each doc in our new list

    idx = list(range(len(new_doc)))
    D=dict.fromkeys(idx)
    D.update(zip(idx,new_doc))
    sub_docs = D
    
    vocab = set()
    n_nonzero = 0
    
    #
    # make sub_docs into a single document term matrix
    #
    
    for docterms in sub_docs.values():
        unique_terms = set(docterms)    # all unique terms of this doc
        vocab |= unique_terms           # set union: add unique terms of this doc
        n_nonzero += len(unique_terms)  # add count of unique terms in this doc
        

    docnames = list(sub_docs.keys())

    docnames = np.array(docnames)
    vocab = np.array(list(vocab))  
    vocab_sorter = np.argsort(vocab)  
    ndocs = len(docnames)
    nvocab = len(vocab)

    data = np.empty(n_nonzero, dtype=np.intc)     # all non-zero term frequencies at data[k]
    rows = np.empty(n_nonzero, dtype=np.intc)     # row index for kth data item (kth term freq.)
    cols = np.empty(n_nonzero, dtype=np.intc)     # column index for kth data item (kth term freq.)

    # current index in the sparse matrix data 
    ind = 0 

    # go through all documents, get integer encodings of vocab, and create document term matrix 
    # of subdocs for this post
    for docname, terms in sub_docs.items():

        term_indices = vocab_sorter[np.searchsorted(vocab, terms, sorter=vocab_sorter)]

        # count the unique terms of the document and get their vocabulary indices
        uniq_indices, counts = np.unique(term_indices, return_counts=True)
        n_vals = len(uniq_indices)  # = number of unique terms
        ind_end = ind + n_vals  # slice that we will fill with data

        # term frequencies 
        data[ind:ind_end] = counts                  
        cols[ind:ind_end] = uniq_indices
        # get the document index for the document name
        doc_idx = np.where(docnames == docname)     
        rows[ind:ind_end] = np.repeat(doc_idx, n_vals)  # save it as repeated value

        # resume with next document, since we have to collect vocab from all subdocs 
        ind = ind_end  
    
    # build the matrix for current list of sub_docs
    dtm = coo_matrix((data, (rows, cols)), shape=(ndocs, nvocab), dtype=np.intc)
    
    # fit the model
    model.fit(dtm)
    
    # get the list of topics we created
    topic_word = model.topic_word_  
    
    #use regex to strip off the word for this doc, then get integer lable suitable for the classifiers
    red_name = re.findall("(\D+)", postname)
    sub_names = np.array(['xkcd', 'scala', 'surfing', 'MechanicalKeyboards', 'zelda', 'optometry', 'Cameras', 'Genealogy'])
    vocab_sorter = np.argsort(name)
    lable = vocab_sorter[np.searchsorted(sub_names, red_name, sorter=vocab_sorter)]


    new_row = pd.DataFrame([lable])

    # append the results to our output file
    for i, topic_dist in enumerate(topic_word):
      topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]

      df2 = pd.DataFrame([topic_words]) 
      new_row = pd.concat([new_row, df2], axis=1) 
    
    new_row.to_csv('results.csv', header=False, index=False, mode='a')
    
    iteration += 1

INFO:lda:n_documents: 31
INFO:lda:vocab_size: 145
INFO:lda:n_words: 231
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -1496
INFO:lda:<10> log likelihood: -1403
INFO:lda:<20> log likelihood: -1426
INFO:lda:<30> log likelihood: -1421
INFO:lda:<40> log likelihood: -1407
INFO:lda:<49> log likelihood: -1413
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 96
INFO:lda:n_words: 128
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50


ITERATION __________________________:  0
ITERATION __________________________:  1


INFO:lda:<0> log likelihood: -784
INFO:lda:<10> log likelihood: -755
INFO:lda:<20> log likelihood: -770
INFO:lda:<30> log likelihood: -754
INFO:lda:<40> log likelihood: -754
INFO:lda:<49> log likelihood: -757
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 70
INFO:lda:n_words: 179
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -951
INFO:lda:<10> log likelihood: -869
INFO:lda:<20> log likelihood: -869
INFO:lda:<30> log likelihood: -870
INFO:lda:<40> log likelihood: -858
INFO:lda:<49> log likelihood: -865
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 131


ITERATION __________________________:  2
ITERATION __________________________:  3


INFO:lda:n_words: 260
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -1521
INFO:lda:<10> log likelihood: -1428
INFO:lda:<20> log likelihood: -1434
INFO:lda:<30> log likelihood: -1421
INFO:lda:<40> log likelihood: -1398
INFO:lda:<49> log likelihood: -1409
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 147
INFO:lda:n_words: 192
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -1255
INFO:lda:<10> log likelihood: -1214
INFO:lda:<20> log likelihood: -1221
INFO:lda:<30> log likelihood: -1215
INFO:lda:<40> log likelihood: -1217
INFO:lda:<49> log likelihood: -1226
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 108
INFO:lda:n_words: 160
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50


...

ITERATION __________________________:  1788


INFO:lda:<49> log likelihood: -1344
INFO:lda:n_documents: 31
INFO:lda:vocab_size: 80
INFO:lda:n_words: 109
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -663
INFO:lda:<10> log likelihood: -627
INFO:lda:<20> log likelihood: -631
INFO:lda:<30> log likelihood: -630
INFO:lda:<40> log likelihood: -626
INFO:lda:<49> log likelihood: -637


ITERATION __________________________:  1789


INFO:lda:n_documents: 31
INFO:lda:vocab_size: 113
INFO:lda:n_words: 136
INFO:lda:n_topics: 2
INFO:lda:n_iter: 50
INFO:lda:<0> log likelihood: -874
INFO:lda:<10> log likelihood: -853
INFO:lda:<20> log likelihood: -855
INFO:lda:<30> log likelihood: -856
INFO:lda:<40> log likelihood: -850
INFO:lda:<49> log likelihood: -853


ITERATION __________________________:  1790
